<a href="https://colab.research.google.com/github/Mhobo/Dissertation/blob/main/SqueezeNetCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import numpy as np
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import random as rnd
from scipy.stats import bernoulli
import time
import copy
import matplotlib.pyplot as plt
import pickle

batch_size = 125
num_workers = 2


train_dataset = torchvision.datasets.CIFAR10(root='data/',
                                             train=True, 
                                             transform=transforms.ToTensor(),  #replace with = transform after
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='data/',
                                            train=False,#need comma here to get transforms back
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                            batch_size= batch_size, 
                                            shuffle=True, num_workers = num_workers)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                            batch_size= batch_size, 
                                            shuffle=False, num_workers = num_workers)



Files already downloaded and verified


In [10]:
# Corrupts Labels
def Changer(p = 0.5):

#  Changed = range(0,6000,Ratio)
  Changed_dataset = copy.deepcopy(train_dataset)
# set new labels that are wrong
  New_Values = []
  
  for i in range(len(Changed_dataset.targets)): #[Changed]
    if bernoulli.rvs(p) == 1:
      continue
    else:

      Label_Options = list(range(0,10))
      Label_Options.remove(Changed_dataset.targets[i]) #Changed[i]
      New_Value = rnd.choice(Label_Options)
      Changed_dataset.targets[i] = New_Value
      New_Values.append(New_Value)


  return Changed_dataset


In [17]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.functional as F
import numpy as np
import torch.optim as optim
import math

class fire(nn.Module):
    def __init__(self, inplanes, squeeze_planes, expand_planes):
        super(fire, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, squeeze_planes, kernel_size=1, stride=1)
        self.bn1 = nn.BatchNorm2d(squeeze_planes)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(squeeze_planes, expand_planes, kernel_size=1, stride=1)
        self.bn2 = nn.BatchNorm2d(expand_planes)
        self.conv3 = nn.Conv2d(squeeze_planes, expand_planes, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(expand_planes)
        self.relu2 = nn.ReLU(inplace=True)

        # using MSR initilization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.in_channels
                m.weight.data.normal_(0, math.sqrt(2./n))

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        out1 = self.conv2(x)
        out1 = self.bn2(out1)
        out2 = self.conv3(x)
        out2 = self.bn3(out2)
        out = torch.cat([out1, out2], 1)
        out = self.relu2(out)
        return out


class SqueezeNet(nn.Module):
    def __init__(self):
        super(SqueezeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 96, kernel_size=3, stride=1, padding=1) # 32
        self.bn1 = nn.BatchNorm2d(96)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2) # 16
        self.fire2 = fire(96, 16, 64)
        self.fire3 = fire(128, 16, 64)
        self.fire4 = fire(128, 32, 128)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2) # 8
        self.fire5 = fire(256, 32, 128)
        self.fire6 = fire(256, 48, 192)
        self.fire7 = fire(384, 48, 192)
        self.fire8 = fire(384, 64, 256)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2) # 4
        self.fire9 = fire(512, 64, 256)
        self.conv2 = nn.Conv2d(512, 10, kernel_size=1, stride=1)
        self.avg_pool = nn.AvgPool2d(kernel_size=4, stride=4)
        self.softmax = nn.LogSoftmax(dim=1)
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.in_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()


    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool1(x)
        x = self.fire2(x)
        x = self.fire3(x)
        x = self.fire4(x)
        x = self.maxpool2(x)
        x = self.fire5(x)
        x = self.fire6(x)
        x = self.fire7(x)
        x = self.fire8(x)
        x = self.maxpool3(x)
        x = self.fire9(x)
        x = self.conv2(x)
        x = self.avg_pool(x)
        x = self.softmax(x)
        return x,x

def fire_layer(inp, s, e):
    f = fire(inp, s, e)
    return f

def squeezenet(pretrained=False):
    net = SqueezeNet()
    # inp = Variable(torch.randn(64,3,32,32))
    # out = net.forward(inp)
    # print(out.size())
    return net

# if __name__ == '__main__':
#     squeezenet()

In [4]:
#for epoch in range(num_epochs):
def trainCNN(P_DML_ss = 0,P_DML = 1, P_SS =1, ss = False):  
  net.train()
  net2.train()

  # total = 0
  # correct = 0
  # correct2 = 0

# remove enumerate and _? 
  for i, (images, labels) in enumerate(train_loader):
    
    images = images.to(device)
    labels = labels.to(device)

    guesses, _ = net.forward(images)
    guesses = guesses.view(batch_size,10)
    loss = loss_fn(guesses,labels)

    guesses2, _ = net2.forward(images)
    guesses2 = guesses2.view(batch_size,10)    
    loss2 = loss_fn(guesses2,labels)
    
    #DML_Loss
    # Aleternate depending on epoch?
    loss_DML = P_DML*loss_fn(guesses,guesses2.softmax(dim=1).detach())
    loss += loss_DML

    loss_DML2 = P_DML*loss_fn(guesses2,guesses.softmax(dim=1).detach())
    loss2 += loss_DML2

    if ss == True: # Rotation Predictions
      bx = images
      
      curr_batch_size = bx.size(0)
      by_prime = torch.cat((torch.zeros(bx.size(0)), torch.ones(bx.size(0)),
                                  2*torch.ones(bx.size(0)), 3*torch.ones(bx.size(0))), 0).long()
      
      bx = images
      bx = torch.cat((bx, torch.rot90(bx,1,[2,3]),torch.rot90(bx, 2,[2,3]), torch.rot90(bx, 3,[2,3])), 0)
      
      bx, by_prime = bx.to(device), by_prime.to(device)

      _, pen = net(bx)
      _, pen2 = net2(bx)

      ss_guesses = net.rot_pred(pen)
      ss_guesses2 = net2.rot_pred(pen2)

      ss_loss = P_SS*loss_fn(ss_guesses, by_prime)
      ss_loss2 = P_SS*loss_fn(ss_guesses2, by_prime)

      loss_DML_ss = P_DML_ss*loss_fn(ss_guesses,ss_guesses2.softmax(dim=1).detach())
      loss += loss_DML_ss

      loss_DML2_ss = P_DML_ss*loss_fn(ss_guesses2,ss_guesses.softmax(dim=1).detach())
      loss2 += loss_DML2_ss


      loss += ss_loss
      loss2 += ss_loss2



# Usually when you get a gradient in a step and then recalculate (for next step) it accumulates the gradient
# This is actually useful for batch gradient descent when you can't do everything at once, but here we aren't memory constrained and want them to start at 0
    optimizer.zero_grad() 
    
    loss.backward()
    loss2.backward()
    optimizer.step()

# loss.item just takes the loss written as a tensor and returns as  a float

  scheduler.step(loss)
#     Decay learning rate
#    if (epoch+1) % 20 == 0:
#        curr_lr /= 3
#        update_lr(optimizer, curr_lr)


  return loss


In [5]:
def testRes(test_loader = test_loader):
  net.eval()
  net2.eval()
  with torch.no_grad():
      correct = 0
      total = 0

      correct2 = 0
      total2 = 0      

      for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        guesses,_ = net.forward(images)
        guesses = guesses.view(batch_size,10)
        _, predicted = torch.max(guesses.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        test_accuracy = correct / total

        guesses2,_ = net2(images)
        _, predicted2 = torch.max(guesses2.data, 1)
        total2 += labels.size(0)
        correct2 += (predicted2 == labels).sum().item()
  return test_accuracy

In [21]:
torch.manual_seed(0)
device = torch.device('cuda')
num_epochs = 10
learning_rate = 0.1
loss_fn = nn.CrossEntropyLoss() # Also an nn.module

net_safe = squeezenet()
net2_safe = squeezenet()

net_safe.rot_pred  = nn.Linear(256, 4)
net2_safe.rot_pred = nn.Linear(256, 4)

net_safe.to(device)
net2_safe.to(device)
            
            
optimizer = torch.optim.SGD(list(net.parameters()) +list(net2.parameters()), lr= learning_rate) # An optimiser object
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience = 5)
        

In [18]:
# Matrix of what I want to train:
Corruption = [1,0.5,0.2]
DML = [0,1]

Type = ['Unchanged','DML']

In [ ]:
All_accs = []
for k in range(3):
    for j in range(2):
        for i in range(3):
            
            p = Corruption[i]
            Changed_dataset = Changer(p)
            Changed_loader = torch.utils.data.DataLoader(dataset=Changed_dataset,
                                                batch_size= batch_size, 
                                                shuffle=True, num_workers = num_workers)
    
            net = copy.deepcopy(net_safe)
            net2 = copy.deepcopy(net2_safe)
            
            net.to(device)
            net2.to(device)
            
            
            
            optimizer = torch.optim.SGD(list(net.parameters()) +list(net2.parameters()), lr= learning_rate) # An optimiser object
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience = 5)
              
            loss = np.array([])
            acc = np.array([])
              
            for epoch in range(num_epochs):
                  
                begin_epoch = time.time()\
                  
                new_train_loss = trainCNN(P_DML_ss= 0, P_DML = DML[j], P_SS = 0,ss = False)
                
                
                loss = np.append(loss,new_train_loss.cpu().detach().numpy())
                new_acc = testRes()
                acc = np.append(acc, new_acc)
              
                print('Epoch', epoch, '| Time Spent:', round(time.time() - begin_epoch, 2), 'Train_Loss:',new_train_loss, 'Test_Accuracy1:', new_acc)
            print(Corruption[i])
            print(Type[j])
            print(acc)
            All_accs.append((acc,Type[j]))

Epoch 0 | Time Spent: 31.39 Train_Loss: tensor(1.1737, device='cuda:0', grad_fn=<AddBackward0>) Test_Accuracy1: 0.493
Epoch 1 | Time Spent: 31.22 Train_Loss: tensor(1.0402, device='cuda:0', grad_fn=<AddBackward0>) Test_Accuracy1: 0.5818
Epoch 2 | Time Spent: 31.34 Train_Loss: tensor(0.9245, device='cuda:0', grad_fn=<AddBackward0>) Test_Accuracy1: 0.5843
Epoch 3 | Time Spent: 31.31 Train_Loss: tensor(0.8743, device='cuda:0', grad_fn=<AddBackward0>) Test_Accuracy1: 0.628
Epoch 4 | Time Spent: 31.29 Train_Loss: tensor(0.8751, device='cuda:0', grad_fn=<AddBackward0>) Test_Accuracy1: 0.6449
Epoch 5 | Time Spent: 31.22 Train_Loss: tensor(0.7830, device='cuda:0', grad_fn=<AddBackward0>) Test_Accuracy1: 0.6979
Epoch 6 | Time Spent: 31.3 Train_Loss: tensor(0.6983, device='cuda:0', grad_fn=<AddBackward0>) Test_Accuracy1: 0.6465
Epoch 7 | Time Spent: 31.43 Train_Loss: tensor(0.8219, device='cuda:0', grad_fn=<AddBackward0>) Test_Accuracy1: 0.6704
Epoch 8 | Time Spent: 31.38 Train_Loss: tensor(0.61